# BST 261: Final Project

## Subtitle


### 1. Overview 


#### 1.1 Motivation

Medical coding is a multibillion dollar industry which is highly labor intensive and prone to error. This presents an opportunity for Natural Language Processing (NLP).

ICD-9 codes were created by CMS (Centers for Medicare and Medicaid Services) to standardize the way in which patients health outcomes were categorized and tracked over time. They can entered into a patient's electronic health record and can be used for diagnostic, billing and reporting purposes. Currently identifying an ICD-9 code is an manual process, which is slow, expensive, and error prone. Creating a model that could automate the prediction of ICD-9 codes off of doctor's notes would be very beneficial.

The specific scope of our project is to predict the 5 most common ICD-9 codes from doctor discharge notes collected from ICU stays between 2001 and 2011 dates at the Beth Israel Deaconess Medical Center.

#### 1.2 Related Work

Automated ICD-9 codes assignment has been studied since 1990. Previous work has been focussed on pattern matching, rule base systems, or supervised classification methods such as Logistic Regression, k-NN and Support Vector Machines.

Manual methods and supervised have shown good performance for specific sets of codes and data sets, however these do not generalize well.

Deep learning has potential to overcome the limitations of traditional machine learning and rule based systems by eliminating the task of describing explicit features or rules. Deep learning models looking at classifying the top 10 ICD-9 codes achieved relatively low performance (F1score: 0.37) suggesting that there is considerable room for improvement in both word representation and model architecture applied.

### 2. Data

We got our data from the MIMIC-III Critical Care Database, which contains data on over 40,000 ICU patients at Beth Isreal Deaconess Medical Center from 2001-2012. Though this database contains massive amounts of data on patient demographics, vital signs, procedures, practitioner notes, we focused on patient discharge notes and their associated ICD-9 codes.

#### 2.1 Data Extraction

We used the following three tables from the MIMIC-III Database:
    
* `D_ICD_Diagnoses`: this table contained the definitions & names of the ICD-9 codes. This was used so we could interpret which ICD-9 codes we were using.
* `DIAGNOSES_ICD`: this table gave us the ICD-9 codes given to each patient at each visit. There is one row for each ICD-9 code for each patient's `HADM_ID` (admission ID).
* `NOTEEVENTS`: this table gives has all the notes for the patients in the database. Notes also include those for echo, ECG, and radiology reports. We only used discharge notes for this project.

To simplify our network, we are only attempting to predict a subset of ICD-9 codes. We ran some numbers on the top (most common) ICD-9 codes in the code chunk below and decided to use the top 5 codes that didn't contain the "V" prefix. The "V" stands for Supplementary Classification of Factors Influencing Health Status and Contact with Health Services. Since they indicate conditions that influence care but do not necessarily represent the outcome of the visit, we decided to exclude those codes from our project.

Note: because the entire table was so large, we ran a SQL query to get the counts of each ICD-9 code, which was saved into the csv file titled 'top100_dx.csv'.

In [20]:
import pandas as pd

# read in data
top_codes = pd.read_csv('/Users/katherine/Desktop/BST 261/Final Project/data/top100_dx.csv',header=None)
diags = pd.read_csv('/Users/katherine/Desktop/BST 261/Final Project/data/D_ICD_Diagnoses.csv')

# merge tables
df=pd.merge(top_codes,diags,left_on=0, right_on='ICD9_CODE',how='left')[:30]
df=df.rename(columns={0:"ICD", 1:"count"})

# removed ICD-9 codes starting with 'V'
df=df[~df['ICD'].str.contains("V")]
print(df.iloc[:5,1:])

ICD_CODES = ["4019","4280","42731","41401","5849"]

   count  ROW_ID ICD9_CODE               SHORT_TITLE  \
0  20703  4304.0      4019          Hypertension NOS   
1  13111  4473.0      4280                   CHF NOS   
2  12891  4462.0     42731       Atrial fibrillation   
3  12429  4374.0     41401  Crnry athrscl natve vssl   
4   9119  5908.0      5849  Acute kidney failure NOS   

                                          LONG_TITLE  
0                 Unspecified essential hypertension  
1              Congestive heart failure, unspecified  
2                                Atrial fibrillation  
3  Coronary atherosclerosis of native coronary ar...  
4                  Acute kidney failure, unspecified  


Once we found the five ICD-9 codes we wanted to predict, the next step was to subset the data and do the cleaning & preprocessing to get it ready to be inputted into our network.

#### 2.2 Data Cleaning/Preprocessing

##### 2.2.1 Fetch Embeddings & Notes Input Data

In [ ]:
word_vectors = Word2Vec.load('./data/w2v_embeddings')
# word_vectors = FastText.load('fasttext_embeddings')
embedding_dim = word_vectors.wv.vectors.shape[1]

'''
**ONLY RUN IF NEED TO RE-PROCESS TEXT**
Read in notes and pre-process text
Also fetch HADM_IDs
'''
hadm_id = []
input_notes = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        if note[6] == 'Discharge summary':
            hadm_id.append(note[2])
            note = preprocess_text2(note[-1])
            input_notes.append(note)
            
'''
Save processed input notes
'''
with open("processed_input_notes.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(input_notes)
    
'''
Only fetch HADM_IDs
'''
hadm_id = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        if note[6] == 'Discharge summary':
            hadm_id.append(note[2])
            
'''
Load processed input notes 
'''
input_notes = []
with open("processed_input_notes.csv", "r") as f:
    reader = csv.reader(f)
    for note in reader:
        input_notes.append(note)
deleted_dupes = 0

The distribution of the note lengths is as follows:

<img src="http://drive.google.com/uc?export=view&id=1MF7HEypxGZ9yqgFUvZFsXgMMJrFPMrTc" alt="Note Length" style="width: 450px;"/>

This prompted us to choose a sequence length of 3000 per summary for our purpose, considering a good analytical potential, computational resource availability and time constraint.

The frequency of the ICD-9 codes in our data is as follows:

<img src="http://drive.google.com/uc?export=view&id=1gBOmHQLXDDrjniphAcQtz9cTs6Opk9D8" alt="Codes" style="width: 450px;"/>

We narrowed down to the top 4 ICD-9 codes based on this distribution

##### 2.2.2 Labels Input Data

We are trying to predict the top 5 ICD-9 codes, so the outcomes are categorical. We'll need to feed 5x1 vectors to the neural network, where each ICD-9 code is represented by either a 0 if the patient was not assigned the code or 1 if the patient was assigned the code. 

We also filtered out any notes that were not discharge summaries since we want to predict ICD-9 codes that were assigned to patients after their ICU visit. There were also many visits that had multiple notes, meaning there were many `HADM_ID`'s with multiple occurrences in the table. Notes sharing the same `HADM_ID` required some extra consideration as well. There were a couple options we considered:

* concatenating/merging notes into 1 large note per `HADM_ID`
* keeping only the lastest note for the patient. The idea behind this was that there may be duplicate information in the notes, but the latest note would likely summarize the patient's condition the best.
* removing these duplicate `HADM_ID`'s all together.

Any admissions (`HADM_ID`'s) that appeared multiple times in the dataset (i.e. visits with multiple discharge summary notes) were dropped to simplify the problem. 

In [ ]:
df1 = pd.read_csv(DIAGNOSES_ICD)
df2 = pd.read_csv(NOTEEVENTS)

# Translate ICD codes to indicator variables
dummy = pd.get_dummies(df1['ICD9_CODE'])[ICD_CODES]
# Append indicator var columnns to original ICD df
dummy_combined = pd.concat([df1, dummy], axis=1)
# Combine by HADM_ID and drop columns
dummy_combined = dummy_combined.groupby(['HADM_ID'], as_index=False).sum().drop(['ROW_ID','SUBJECT_ID','SEQ_NUM'], axis = 1)

# now join the two tables together 
df_final = pd.merge(df2, dummy_combined,left_on="HADM_ID", right_on='HADM_ID', how='left')
# Filter by discharge summary
df_final = df_final[df_final['CATEGORY'] == 'Discharge summary']
# removed any hadmid that have more than one entry in database
df_final = df_final.drop_duplicates(subset = "HADM_ID", keep = False)
df_final = df_final.drop(['ROW_ID', 'SUBJECT_ID', 'CHARTDATE', 
                          'CHARTTIME', 'STORETIME', 'CATEGORY',
                         'DESCRIPTION', 'CGID', 'ISERROR'], axis = 1)

##### 2.2.3 Subsampling the Data

We randomly subsampled the data to allow us to train our neural network faster under our time constraints. The data was randomly sampled to ensure that the network was trained on some data that did not have any of the top 5 ICD-9 codes.

In [ ]:
# random sample of the data
sub_df_final = df_final.sample(10000)

input_notes = []
hadm_id = []
y = []
for i in range(len(sub_df_final)):
    y.append(list(map(int, sub_df_final.iloc[i, 2:].tolist())))
    input_notes.append(preprocess_text2(sub_df_final.iloc[i, 1]))
    hadm_id.append(sub_df_final.iloc[i, 0])
y = np.array(y)

'''
Train/test split
'''
x_train, x_test, y_train, y_test = train_test_split(input_notes, y, test_size=0.2)

from keras.preprocessing.text import Tokenizer
'''
Takes words and converts to indices (which then correspond to vectors in
the embedding models)
'''
#max_words = len(word_vectors.wv.vocab)
max_words = 15000
token = Tokenizer(max_words)
token.fit_on_texts(input_notes)
vocab_size = max_words + 1

sequences = token.texts_to_sequences(x_train)
test_sequences = token.texts_to_sequences(x_test)

'''
Convert to padded sequences
'''
from keras.preprocessing.sequence import pad_sequences
seq_len = 3000
X = pad_sequences(sequences, maxlen=seq_len)
X_test = pad_sequences(test_sequences, maxlen=seq_len)

embeddings_index = {}
vocab = token.word_index.keys()
for word in vocab:
    if word in word_vectors.wv.vocab:
        coefs = np.asarray(word_vectors.wv[word], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

word_index = token.word_index
embedding_matrix = np.zeros((vocab_size , embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
            
num_classes = y_train.shape[1]

### 3. Model Architecture

##### Gated Recurrent Network (GRU)

GRU's are a relatively new model similar to LSTM's in that it solves vanishing gradient problem from standard RNNs. GRU's combine the 'forget' & 'input' gates used in LSTM to an 'update' gate. The network then uses an 'update' and 'reset' gate to decide what information is kept as the model is being trained.

It is said that GRU's and LSTM's usually have similar performance, but GRU's tend to be faster to train and easier to use. This is why we decided to use GRU's first.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
from keras.models import load_model

m = load_model('model_20180501_liam')
m.summary()

**Liam GRU**

In [ ]:
import csv
import re
import string
from timeit import default_timer as timer
import gensim
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, Input, Dropout, Bidirectional, GaussianNoise
from keras.layers.recurrent import LSTM, GRU

import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf


#######################################################
'''
Edit these constants to change important things in code below
'''
NOTEEVENTS = "./data/NOTEEVENTS.csv"
DIAGNOSES_ICD = "./data/DIAGNOSES_ICD.csv"
EMBEDDING_MODEL = 'w2v' # 'w2v' or 'ft'
ICD_CODES = ["4019","4280","42731", "41401", "5849"] # ICD codes of interest
SEQ_LEN = 3000 # Max length of note
MAX_WORDS = 50000

SUBSAMPLE_SIZE = 20000
TEST_SET_FRACTION = 0.2
BATCH_SIZE = 128
TRAINING_EPOCHS = 5
MODEL_SAVE_PATH = './models/model_20180502_01_liam' # Path to save trained model to
#######################################################


def clean_string1(text):
    """
    """
    text = text.strip().lower().replace('-', '_').replace('.', '_').replace(' ', '_').rstrip('_')
    return text


def preprocess_text2(query):
    """
    """
    query = re.sub('\d+|\d+\.\d+', '[NUM]', query)
    query = re.sub('(\[\*\*.*?\*\*\])', '[PHI]', query)
    query = query.strip('"').strip('?').strip("'").strip('(').strip(')').strip(':')
    query = re.sub('['+'!"#$%&\'()*+,-./:;<=>?@\\^`{|}~'+']', '', query)
    word_list = query.split()
    word_list = [clean_string1(word) for word in word_list]
    return word_list


if EMBEDDING_MODEL == 'w2v':
    word_vectors = Word2Vec.load('./data/w2v_embeddings')
else:
    word_vectors = FastText.load('./data/fasttext_embeddings')
embedding_dim = word_vectors.wv.vectors.shape[1]


df1 = pd.read_csv(DIAGNOSES_ICD)
df2 = pd.read_csv(NOTEEVENTS)
# Translate ICD codes to indicator variables
dummy = pd.get_dummies(df1['ICD9_CODE'])[ICD_CODES]
# Append indicator var columnns to original ICD df
dummy_combined = pd.concat([df1, dummy], axis=1)
# Combine by HADM_ID and drop columns
dummy_combined = dummy_combined.groupby(['HADM_ID'], as_index=False).sum().drop(['ROW_ID','SUBJECT_ID','SEQ_NUM'], axis = 1)
#now join the two tables together
df_final = pd.merge(df2, dummy_combined,left_on="HADM_ID", right_on='HADM_ID', how='left')
# Filter by discharge summary
df_final = df_final[df_final['CATEGORY'] == 'Discharge summary']
# removed any hadmid that have more than one entry in database
df_final = df_final.drop_duplicates(subset = "HADM_ID", keep = False)
df_final = df_final.drop(['ROW_ID', 'SUBJECT_ID', 'CHARTDATE',
                          'CHARTTIME', 'STORETIME', 'CATEGORY',
                         'DESCRIPTION', 'CGID', 'ISERROR'], axis = 1)
#random sample of the data
sub_df_final = df_final.sample(SUBSAMPLE_SIZE)

# Read in and process data
input_notes = []
hadm_id = []
y = []
for i in range(len(sub_df_final)):
    y.append(list(map(int, sub_df_final.iloc[i, 2:].tolist())))
    input_notes.append(preprocess_text2(sub_df_final.iloc[i, 1]))
    hadm_id.append(sub_df_final.iloc[i, 0])
y = np.array(y)

'''
Train/test split
'''
x_train, x_test, y_train, y_test = train_test_split(input_notes, y, test_size=TEST_SET_FRACTION)

from keras.preprocessing.text import Tokenizer
'''
Takes words and converts to indices (which then correspond to vectors in
the embedding models)
'''
#max_words = len(word_vectors.wv.vocab)
max_words = MAX_WORDS
token = Tokenizer(max_words)
token.fit_on_texts(input_notes)
vocab_size = max_words + 1

sequences = token.texts_to_sequences(x_train)
test_sequences = token.texts_to_sequences(x_test)

'''
Convert to padded sequences
'''
from keras.preprocessing.sequence import pad_sequences
seq_len = SEQ_LEN
X = pad_sequences(sequences, maxlen=seq_len)
X_test = pad_sequences(test_sequences, maxlen=seq_len)

embeddings_index = {}
vocab = token.word_index.keys()
for word in vocab:
    if word in word_vectors.wv.vocab:
        coefs = np.asarray(word_vectors.wv[word], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

word_index = token.word_index
embedding_matrix = np.zeros((vocab_size , embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        embedding_vector = embeddings_index.get(word)
          if embedding_vector is not None:
              # words not found in embedding index will be all-zeros.
              embedding_matrix[i] = embedding_vector

num_classes = y_train.shape[1]

import keras.backend as K

def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def calculate_class_weights(Y):
    '''
    input: np array of labels
    output: dict of multilabel class weights
    '''
    l_weights = []
    c_weights = []
    for i in range(y.shape[1]):
        neg = len(y[y[:,i] == 0, i])
        pos = len(y) - neg
        neg_ratio = neg / pos
        l_weights.append(neg_ratio)
        c_weights.append({0: 1, 1: neg_ratio})
    return tf.constant(l_weights), c_weights

loss_weights, class_weights = calculate_class_weights(y_train)

# config = K.tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = K.tf.Session(config=config)

def _to_tensor(x, dtype):
    """Convert the input `x` to a tensor of type `dtype`.
    # Arguments
        x: An object to be converted (numpy array, list, tensors).
        dtype: The destination type.
    # Returns
        A tensor.
    """
    return tf.convert_to_tensor(x, dtype=dtype)


def weighted_binary_crossentropy(target, output, from_logits=False):
  """Binary crossentropy between an output tensor and a target tensor.
  Arguments:
      output: A tensor.
      target: A tensor with the same shape as `output`.
      from_logits: Whether `output` is expected to be a logits tensor.
          By default, we consider that `output`
          encodes a probability distribution.
  Returns:
      A tensor.
  """
  # Note: nn.softmax_cross_entropy_with_logits
  # expects logits, Keras expects probabilities.
      if not from_logits:
        # transform back to logits
        epsilon = _to_tensor(K.epsilon(), output.dtype.base_dtype)
        output = tf.clip_by_value(output, epsilon, 1 - epsilon)
        output = tf.log(output / (1 - output))
    return tf.nn.weighted_cross_entropy_with_logits(target, output, pos_weight=loss_weights)


from keras.layers import Lambda
ClipLayer = Lambda(lambda x: K.clip(x, min_value=0.01, max_value=0.99))

## Build the model ##
input = Input(shape=(seq_len,))
x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False)(input)
x = GaussianNoise(0.75)(x)
x = Bidirectional(GRU(units=128, recurrent_dropout=0.2, dropout=0.2, activation='relu', return_sequences=True))(x)
x = Bidirectional(GRU(units=128, recurrent_dropout=0.2, dropout=0.2, activation='relu'))(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='sigmoid')(x)
x = ClipLayer(x)
model = Model(input, x)

model.compile(optimizer='adam',
              loss=weighted_binary_crossentropy,
              weighted_metrics=['binary_accuracy', precision, recall])

model.fit(X, y_train,
          epochs=TRAINING_EPOCHS,
          batch_size=BATCH_SIZE,
          class_weight=class_weights)

model.save(MODEL_SAVE_PATH)

model.predict(X_test, y_test)

### Discussion

Our model initially produced an accuracy of 74%. However, it convereged very quickly and did not improve with additional training. We were skeptical of our high accuracy, given our slightly imbalanced dataset, so we decided to look at a different metric - Precision/Recall. We generated a recall metric < 0.01, indicating that our model was simply predicting “0” for each diagnosis for nearly every patient. 

We implemented changes in our network architecture to try to improve recall, which did see some light, but the loss quickly converged around 1.03. We suspect this to have been caused by a vanishing gradient. GRUs are structurally fit towards but tackling the vanishing gradient problem, but from our approach to the problem, we believe that it is still affecting the outcome in spite of the model being based on a GRU. A further analysis is required to identify the crux of the problem.

### Future Work 

#### One-Dimensional CNN
A 1-D CNN can extract key features of the data. We can expect this model to have vanishing gradient issues (the model would likely be unable to catch long term meaning throughout the notes) but the CNN may be better at picking up keywords. Implementing this would require more computational resources and time.

#### Additional Considerations
We had to limit the vector length of out discharge summaries due to computational resource and time limitations. A less restrictive trimming of word vectors for each discharge note could have possibly improved the results. Also,
a choice of more ICD-9 codes as potential prediction classes to tend towards the original data distribution
